   # Fairness

In [2]:
import numpy as np
import pandas as pd
import pickle
 
import datetime
dte = datetime.datetime.now()
dte = dte.strftime("%Y%m%d")

# import custom functions
import sys
sys.path.append('../../onc_functions')

from fairness import get_fairness_assessment

print('python-' + sys.version)

python-3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


# Calculate Fairness Assessment
- get the true y-values and the y_prediction_probability from the model
- get details to evaluate for fairness

In [3]:
# load results from the model
pred_df = pd.read_csv('./roc_auc/2021_xgb_nonimputed_y_proba.csv')
pred_df.drop(columns=['Unnamed: 0'],inplace=True)
pred_df.head()

,score,y,usrds_id
0,0.580939,1,31089.0
1,0.399797,0,34521.0
2,0.339976,0,46751.0
3,0.274023,0,50506.0
4,0.046709,0,54985.0


In [4]:
with open('../complete_fairness_data.pickle', 'rb') as f:
        dataset = pickle.load(f)
dataset.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset
0,2969676.0,0,72.0,2.0,1.0,2.0,2.0,8
1,2969869.0,1,73.0,2.0,1.0,2.0,2.0,9
2,2970021.0,0,55.0,2.0,1.0,1.0,1.0,8
3,2970067.0,0,72.0,1.0,1.0,1.0,2.0,9
4,2970686.0,0,49.0,2.0,1.0,2.0,2.0,8


In [5]:
# merge results with fairness details
data = pred_df.merge(dataset, how='left', on=['usrds_id'])
data.head()

,score,y,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset
0,0.580939,1,31089.0,1,80.0,1.0,1.0,2.0,2.0,9
1,0.399797,0,34521.0,0,64.0,2.0,1.0,1.0,2.0,8
2,0.339976,0,46751.0,0,69.0,2.0,1.0,1.0,2.0,8
3,0.274023,0,50506.0,0,73.0,2.0,NaN,1.0,2.0,9
4,0.046709,0,54985.0,0,48.0,1.0,1.0,1.0,2.0,8


In [7]:
fairness = get_fairness_assessment(
                                        data,
                                        y_proba_col_name='score',
                                        y_true_col_name='died_in_90')
fairness.head(39)

,Feature,Value,Count,AUC,TN,FP,FN,TP
0,agegroup,1.0,4340,0.859782,4289,5,45,1
1,agegroup,2.0,12774,0.844446,12523,39,188,24
2,agegroup,3.0,26120,0.848271,25361,178,487,94
3,agegroup,4.0,53564,0.818192,51089,660,1548,267
4,agegroup,5.0,85076,0.799289,78955,1797,3508,816
5,agegroup,6.0,86140,0.785491,74353,4263,5370,2154
6,agegroup,7.0,62193,0.764716,46951,6974,4626,3642
7,agegroup,8.0,15098,0.748486,9194,2936,1235,1733
8,sex,1.0,198347,0.830416,173954,9746,9456,5191
9,sex,2.0,146957,0.818450,128760,7106,7551,3540


In [8]:
#save (for the aucroc evaluated model)
fairness.to_csv('./roc_auc/' + str(dte) + '_xgb_nonimputed_fairness.csv')